In [2]:
import os
import numpy as np
import tensorflow as tf


def load_minibatch(index, var):
    shape = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_shape.npy'))
    for i in range(len(shape)):
        if var.shape[i] == None:
            continue
        elif var.shape[i] != shape[i]:
            raise ValueError('Shape of minibatch and variable are incompatible')
    
    indices = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_indices.npy'))
    values = np.load(os.path.join('DATA', 'MINIBATCHES', f'{index}_values.npy'))
    sparse_tensor = tf.sparse.SparseTensor(indices=indices, values=values, dense_shape=shape)
    var.assign(tf.cast(tf.sparse.to_dense(sparse_tensor), dtype=tf.float32))

In [3]:
var = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])

load_minibatch(0, var)

In [4]:
import os
import json

with open(os.path.join('DATA', 'numberize.json'), 'r') as f:
    numberize = json.load(f)
        
with open(os.path.join('DATA', 'reverse_numberize.json'), 'r') as f:
    reverse_numberize = json.load(f)

In [5]:
def verbalize_sentence(numberized_sentence):
    words = [reverse_numberize[str(word_number)] for word_number in numberized_sentence if word_number != 10001]
    return ' '.join(words)

def verbalize(vectorized_sentences):
    numberized_sentences = tf.argmax(vectorized_sentences, axis=2).numpy()
    return list(map(verbalize_sentence, numberized_sentences))

In [6]:
class BatchGenerator(object):

    def __init__(self, num_batches, hidden_size):
        self.num_batches = num_batches
        self.current_idx = 0
        self.x = tf.Variable(tf.zeros([500, 30, 10002]), dtype=tf.float32, shape=[None, 30, 10002])
        self.hidden_state_0 = tf.zeros((500, hidden_size))
        self.cell_state_0 = tf.zeros((500, hidden_size))
        
    def generate(self):
        while True:
            load_minibatch(self.current_idx, self.x)
            self.current_idx = (self.current_idx + 1) % self.num_batches
            yield [[tf.zeros([500, 10002])] + [self.x[:, i, :] for i in range(29)], self.hidden_state_0, self.cell_state_0], [self.x[:, i, :] for i in range(30)]
            
num_batches = 6807
batch_generator = BatchGenerator(6807, 128).generate()

In [ ]:
from keras import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Activation

model = Sequential()
model.add(LSTM(256, input_shape=(None, 10002), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(10002)))
model.add(Activation('softmax'))

In [ ]:
from keras import Model, Input
from keras.layers import Dense, Activation, Dropout, LSTM, Lambda, Reshape
from keras.optimizers import Adam

sequence_length = 30
vocab_size = 10002
hidden_size = 128

Reshape_cell = Reshape((1, vocab_size)) 
LSTM_cell = LSTM(hidden_size, return_state = True)
Dense_cell = Dense(vocab_size, activation='softmax')

def LSTM_model(sequence_length, hidden_size, vocab_size):
    """
    Arguments:
    sequence_length -- length of input sequences
    hidden_size -- number of hidden units in LSTM cell
    vocab_size -- size of vocabulary in corpus 
    
    Returns:
    a keras model
    """
    
    # Define inputs to LSTM
    X = Input(shape=(sequence_length, vocab_size), name='sequence_input')
    hidden_state_0 = Input(shape=(hidden_size,), name='hidden_state_0')
    cell_state_0 = Input(shape=(hidden_size,), name='cell_state_0')
    
    hidden_state = hidden_state_0
    cell_state = cell_state_0
    outputs = []
    for t in range(sequence_length):
        # Select t-th time slice
        x = Lambda(lambda y: y[:, t, :])(X)
        # Rehapse and apply LSTM_cell to time slice
        x = Reshape_cell(x)
        hidden_state, _, cell_state = LSTM_cell(inputs=x, initial_state=[hidden_state, cell_state])
        # Apply Dense_cell to output of LSTM
        output = Dense_cell(hidden_state)
        # Step 2.E: add the output to "outputs"
        outputs.append(output)
    
    # Step 3: Create model instance
    model = Model(inputs=[X, hidden_state_0, cell_state_0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

model = LSTM_model(sequence_length, hidden_size, vocab_size)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
stuff = batch_generator.__next__()

In [ ]:
[x, a0, c0], y = stuff

In [ ]:
model.fit([x, a0, c0], y, epochs=1)

In [ ]:
from keras import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Activation

model = Sequential()
model.add(LSTM(256, input_shape=(None, 10002), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(10002)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
batches_to_process = 5

model.fit_generator(batch_generator, batches_to_process, 1)

In [ ]:
word_vec = np.zeros((1, 1, 10002))
word_vec[0, 0, 1] = 1

In [ ]:
res = model.predict(word_vec)

In [ ]:
np.argmax(res, axis=2)